In [1]:
#Load related libraries to do analysis
import pandas as pd #To work with dataset
import numpy as np #Math library

In [2]:
df_acquisition = pd.read_table('output/fm2000Q1_acquisition_sample.txt', sep = '|')
df_performance = pd.read_table('output/fm2000Q1_performance_sample.txt', sep = '|')

In [3]:
# Create MOB of all even if account is gone from newer business date
# The use of this methodology depend whether the data gone or not
# The MOB based on tenor

# Choose select columns only to create
df_performance2 = df_performance[['LOAN_ID','ORIG_TRM']]

df_create = pd.concat([pd.DataFrame({'LOAN_ID':row.LOAN_ID,
                                     'MOB':range(0,(row.ORIG_TRM+1)),
                                     'ORIG_TRM':row.ORIG_TRM
                                    },
                                    columns = ['LOAN_ID','MOB','ORIG_TRM'])
                                    for i, row in df_performance2.iterrows()], ignore_index=True)
# a must to remove duplicate due to how above code works
df_create.drop_duplicates(subset = ['LOAN_ID','MOB'], keep = 'first', inplace=True)

# recommended to have, if you know that you only want to track up to MOB 90
#df_create = df_create[df_create.MOB <= 60]


In [4]:
df_performance2 = df_performance[['LOAN_ID','Delq.Status','Loan.Age']]
print(df_performance2.info())
print(df_performance2['Delq.Status'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56932 entries, 0 to 56931
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   LOAN_ID      56932 non-null  int64 
 1   Delq.Status  56919 non-null  object
 2   Loan.Age     56932 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ MB
None
['0' 'X' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' nan
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41']


In [5]:
# Create list of loan ID that ever DPD30+ (kolek 1) or DPD90+ (kolek 2) and the MOB when it happened
df_performance2 = df_performance[['LOAN_ID','Delq.Status','Loan.Age']]
df_performance2 = df_performance2[df_performance2['Delq.Status'] != 'X']
df_performance2['Delq.Status'] = df_performance2['Delq.Status'].astype(float)

# Ever DPD30+
df_ever30 = df_performance2[df_performance2['Delq.Status'] >= 1]
df_ever30_2  = df_ever30.groupby(['LOAN_ID'], as_index = False).agg(
    {
        'Loan.Age':'min'
    }
)
df_ever30_2['EVER30'] = 1
df_ever30_2.rename(columns={'Loan.Age':'DPD30_When'}, inplace = True)

# Ever DPD30+
df_ever90 = df_performance2[df_performance2['Delq.Status'] >= 2]
df_ever90_2  = df_ever90.groupby(['LOAN_ID'], as_index = False).agg(
    {
        'Loan.Age':'min'
    }
)
df_ever90_2['EVER90'] = 1
df_ever90_2.rename(columns={'Loan.Age':'DPD90_When'}, inplace = True)

In [11]:
# Create list of latest date and data
df_performance2 = df_performance[['LOAN_ID','LAST_UPB','Loan.Age','Monthly.Rpt.Prd']]

df_latest  = df_performance2.groupby(['LOAN_ID'], as_index = False).agg(
    {
        'Loan.Age':'max'
    }
)

df_latest_2 = pd.merge(df_latest, df_performance2, left_on = ['LOAN_ID','Loan.Age'], right_on = ['LOAN_ID','Loan.Age'], how = 'left')
df_latest_2.rename(columns={'Loan.Age':'Latest_Loan.Age','Monthly.Rpt.Prd':'Latest_Month.Rp'
                    ,'LAST_UPB':'Latest_LAST_UPB'                    
                    }, inplace = True)

df_latest_2 = pd.merge(df_latest_2, df_ever30_2, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')
df_latest_2 = pd.merge(df_latest_2, df_ever90_2, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')

In [17]:
# Create vintage data with target definition: target Ever DPD90+
df_create_2 = pd.merge(df_create, df_latest_2, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')

df_performance2 = df_performance[['LOAN_ID','Loan.Age','LAST_UPB']]

df_create_2 = pd.merge(df_create_2, df_performance2, left_on = ['LOAN_ID','MOB'], right_on = ['LOAN_ID','Loan.Age'], how = 'left')

# Create EverDPD90 and EverDPD30 by Outstanding tagging

# Create EverDPD90 OS by MOB position
conditions =    [
                ( (df_create_2['EVER90'] > 0) & (df_create_2['Loan.Age'] > df_create_2['DPD90_When']) & (df_create_2['LAST_UPB'] > 0) ),
                ( (df_create_2['EVER90'] > 0) & (df_create_2['Loan.Age'] > df_create_2['DPD90_When']) )
                ]
choices =       [
                df_create_2['LAST_UPB'],
                df_create_2['Latest_LAST_UPB']
                ]
df_create_2['EVER90_OS'] = np.select(conditions,choices, default=0) 

# Create EverDPD30 OS by MOB position
conditions =    [
                ( (df_create_2['EVER30'] > 0) & (df_create_2['Loan.Age'] > df_create_2['DPD30_When']) & (df_create_2['LAST_UPB'] > 0) ),
                ( (df_create_2['EVER30'] > 0) & (df_create_2['Loan.Age'] > df_create_2['DPD30_When']) )
                ]
choices =       [
                df_create_2['LAST_UPB'],
                df_create_2['Latest_LAST_UPB']
                ]
df_create_2['EVER30_OS'] = np.select(conditions,choices, default=0) 

In [19]:
# Create vintage data with acquisition information
df_acquisition2 = df_acquisition[['LOAN_ID','ORIG_CHN','ORIG_AMT']]

df_create_3 = pd.merge(df_create_2, df_acquisition2, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')


In [21]:
# Create vintage report vintage performance by vintage
vintage_channel_excel = 'output/vintage_channel_excel.xlsx'
writer = pd.ExcelWriter(vintage_channel_excel, engine = 'xlsxwriter')

output_ever90_os = pd.crosstab(df_create_3.ORIG_CHN, df_create_3.MOB, values=df_create_3.EVER90_OS, aggfunc='sum')
output_plafond = pd.crosstab(df_create_3.ORIG_CHN, df_create_3.MOB, values=df_create_3.ORIG_AMT, aggfunc='sum')

loss_plafond = output_ever90_os / output_plafond

output_ever90_os.to_excel(writer, sheet_name = 'EVER90_OS_byChannel')
output_plafond.to_excel(writer, sheet_name = 'LIMIT_byChannel')
loss_plafond.to_excel(writer, sheet_name = 'Vintage_byChannel')

writer.save()
writer.close()